In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import spacy
import seaborn as sns
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
!python -m spacy download el_core_news_sm

2023-07-16 12:11:33.185073: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 29.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')


In [3]:
nlp = spacy.load('el_core_news_sm')

In [4]:
cv = CountVectorizer(stop_words= list(nlp.Defaults.stop_words), min_df=0.01, max_df=0.95)

In [5]:
start_page = 1
end_page = 3

start_url = 'https://www.tovima.gr/search/%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82/page/'

test_url = start_url + str(start_page)

test_url

'https://www.tovima.gr/search/%CE%B1%CE%BD%CF%84%CE%B9%CE%B5%CE%BC%CE%B2%CE%BF%CE%BB%CE%B9%CE%B1%CF%83%CF%84%CE%AD%CF%82/page/1'

In [6]:
response = requests.get(test_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [7]:
teaser_articles_list = doc.find('div', {'class': 'column is-relative'}).find_all('div', {'class': 'column is-full wrap-category-row'})

In [8]:
teaser_articles_list

[<div class="column is-full wrap-category-row"><div class="simple-row"> <a class="columns is-mobile is-multiline" href="https://www.tovima.gr/2022/07/30/world/aystria-aytoktonise-giatros-pou-eixe-ginei-stoxos-apeilon-apo-antiemvoliastes/"><div class="column is-2-desktop is-2-tablet is-2-mobile is-full-small"> <time class="line-height-1 mr-3 post-date updated manrope is-size-5 semigrey-c"> 30.07.2022, 13:46 </time></div><div class="column is-6-desktop is-6-tablet is-6-mobile is-full-small"><div class="wrap-row-header-n-desk"><h3 class="o-head f-400 my-0 is-size-2 zonabold"> Αυτοκτόνησε γιατρός που είχε γίνει στόχος αντιεμβολιαστών</h3><p class="is-size-normal mb-0 mt-4 text-ellipsis text-ellipsis-4 grey-c line-height-1-3"> Τραγωδία στην Αυστρία μετά από απειλές αντιεμβολιαστών σε γιατρό..</p><div class="pt-4 category-row-footer"></div></div></div><div class="column is-4-desktop is-4-tablet is-4-mobile is-full-small"><div class="wrap-thumb"><figure class="tile-image thumb-holder" style="

In [9]:
link = teaser_articles_list[0].find('a')['href']

link

'https://www.tovima.gr/2022/07/30/world/aystria-aytoktonise-giatros-pou-eixe-ginei-stoxos-apeilon-apo-antiemvoliastes/'

In [10]:
teasers_list = []

for page_num in range(start_page, end_page+1):
  page_url = start_url + str(page_num)
  response = requests.get(page_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo teaser_articles_list***
  teaser_articles_list = doc.find('div', {'class': 'column is-relative'}).find_all('div', {'class': 'column is-full wrap-category-row'})


  for teaser in teaser_articles_list:
      story_dict = {}
      try:

        #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στο link, ΜΕΤΑ το teaser
        link = teaser.find('a')['href']


        if link.startswith("http"):
          story_dict['url'] = link
        else:
          story_dict['url'] = main_url + link
      except:
        story_dict['url'] = ''
        pass
      teasers_list.append(story_dict)
  time.sleep(1)

In [11]:
tovima_teasers_df = pd.DataFrame(teasers_list)
tovima_teasers_df

,url
0,https://www.tovima.gr/2022/07/30/world/aystria...
1,https://www.tovima.gr/2022/01/24/society/agio-...
2,https://www.tovima.gr/2022/01/23/world/vryksel...
3,https://www.tovima.gr/2022/01/08/world/gallia-...
4,https://www.tovima.gr/2021/12/19/society/masto...
5,https://www.tovima.gr/2021/12/13/society/mitro...
6,https://www.tovima.gr/2021/12/09/inbox/koronoi...
7,https://www.tovima.gr/2021/12/03/vimatv/antiem...
8,https://www.tovima.gr/2021/12/01/politics/voul...
9,https://www.tovima.gr/2021/12/01/opinions/kai-...


In [12]:
tovima_teasers_df.to_csv('/content/temp/tovima_teasers.csv', index=False)

In [13]:
article_url = 'https://www.tovima.gr/2022/07/30/world/aystria-aytoktonise-giatros-pou-eixe-ginei-stoxos-apeilon-apo-antiemvoliastes/'

In [14]:
response = requests.get(article_url)
doc = BeautifulSoup(response.text, 'html.parser')

In [15]:
article = doc.find('main', {'class': 'is-relative post-wrapper maxgridrow art-'})

In [16]:
section = doc.find('div', {'class': 'wrap-post-category'}).find('a').text
section

'Κόσμος '

In [17]:
title = article.find('h1', {'class', "f-400 post-title zonaultra is-size-1-desktop is-size-2-mobile is-size-2-tablet"}).text
title

' Αυστρία: Αυτοκτόνησε γιατρός που είχε γίνει στόχος απειλών από αντιεμβολιαστές'

In [18]:
date = article.find('time', {'class': 'line-height-1 post-date manrope'})['datetime']
wra = article.find('time', {'class': 'line-height-1 post-date manrope lightgrey-c'}).text
datetime = date + ' ' + wra
datetime

'30-07-2022 13:46'

In [19]:
author = article.find('span', {'class': 'vima-author'}).text
author

' Newsroom '

In [20]:
paragraphs = article.find('div', {'class': 'post-body main-content pos-rel article-wrapper'}).find_all('p')
paragraphs

[<p>Οι Αυστριακοί ηγέτες απηύθυναν έκκληση για εθνική ενότητα μετά την αυτοκτονία μιας γιατρού η οποία δεχόταν απειλές θανάτου από ακτιβιστές κατά του εμβολιασμού και από υποστηρικτές θεωριών συνωμοσίας για την πανδημία του κορωνοϊού.<p>«Ας θέσουμε ένα τέλος σε αυτόν τον εκφοβισμό και την τρομολαγνεία. Το μίσος και η μισαλλοδοξία δεν έχουν θέση στην Αυστρία μας», δήλωσε ο πρόεδρος Αλεξάντερ Φαν ντερ Μπέλεν, χαιρετίζοντας τη Λίζα-Μαρία Κέλερμερ ως μια γιατρό που στήριξε τη θεραπεία ανθρώπων, την προστασία τους από ασθένειες έχοντας μια προσεκτική προσέγγιση στην πανδημία.<div class="floated-div;margin:3rem auto" id="300x250_m1" style="display: none"> <script type="text/javascript">googletag.cmd.push(function() { googletag.display('300x250_m1'); });</script></div><p><h2>«Πρώτα στο διαδίκτυο»</h2><p>«Αλλά ορισμένοι άνθρωποι εξοργίστηκαν με αυτό. Και αυτοί οι άνθρωποι την τρόμαξαν, την απείλησαν, πρώτα στο διαδίκτυο και μετά και αυτοπροσώπως…».<p>Το πτώμα της γιατρού — η οποία έδινε συχνά 

In [21]:
p_texts_list = []
paragraphs = article.find('div', {'class': 'post-body main-content pos-rel article-wrapper'}).find_all('p')
for p in paragraphs:
  p_texts_list.append(p.text)
full_text = ' '.join(p_texts_list)
full_text = "".join(full_text.splitlines())
full_text

'Οι Αυστριακοί ηγέτες απηύθυναν έκκληση για εθνική ενότητα μετά την αυτοκτονία μιας γιατρού η οποία δεχόταν απειλές θανάτου από ακτιβιστές κατά του εμβολιασμού και από υποστηρικτές θεωριών συνωμοσίας για την πανδημία του κορωνοϊού.«Ας θέσουμε ένα τέλος σε αυτόν τον εκφοβισμό και την τρομολαγνεία. Το μίσος και η μισαλλοδοξία δεν έχουν θέση στην Αυστρία μας», δήλωσε ο πρόεδρος Αλεξάντερ Φαν ντερ Μπέλεν, χαιρετίζοντας τη Λίζα-Μαρία Κέλερμερ ως μια γιατρό που στήριξε τη θεραπεία ανθρώπων, την προστασία τους από ασθένειες έχοντας μια προσεκτική προσέγγιση στην πανδημία. «Πρώτα στο διαδίκτυο»«Αλλά ορισμένοι άνθρωποι εξοργίστηκαν με αυτό. Και αυτοί οι άνθρωποι την τρόμαξαν, την απείλησαν, πρώτα στο διαδίκτυο και μετά και αυτοπροσώπως…».Το πτώμα της γιατρού — η οποία έδινε συχνά συνεντεύξεις σε μέσα ενημέρωσης για την καταπολέμηση της πανδημίας του κορονοϊού και την προώθηση των εμβολιασμών — βρέθηκε στο γραφείο της στην Άνω Αυστρία χθες Παρασκευή.Μέσα ενημέρωσης ανέφεραν ότι εισαγγελείς είπαν

In [22]:
full_articles_list = []

for article_url in tovima_teasers_df['url']:
  response = requests.get(article_url)
  doc = BeautifulSoup(response.text, 'html.parser')

  #*** ΔΩΣΤΕ ΤΗ ΔΙΚΗ ΣΑΣ ΤΙΜΗ στo article που βρήκατε παραπάνω***
  article = doc.find('main', {'class': 'is-relative post-wrapper maxgridrow art-'})

  full_article_dict = {}

  # website
  full_article_dict['site'] = "tovima.gr"

  # url
  full_article_dict['url'] = article_url

  #section
  try:
    section = doc.find('div', {'class': 'wrap-post-category'}).find('a').text
    full_article_dict['section'] = section
  except:
    full_article_dict['section'] = ''
    pass

  #title
  try:
    title = article.find('h1', {'class', "f-400 post-title zonaultra is-size-1-desktop is-size-2-mobile is-size-2-tablet"}).text
    full_article_dict['title'] = title
  except:
    try:
      title = article.find('h1', {'class', "post-title zonareg is-size-0 is-size-2-mobile is-size-2-tablet f-400"}).text
      full_article_dict['title'] = title
    except:
      full_article_dict['title'] = ''
      pass

  # date
  try:
    date = article.find('time', {'class': 'line-height-1 post-date manrope'})['datetime']
    wra = article.find('time', {'class': 'line-height-1 post-date manrope lightgrey-c'}).text
    datetime = date + ' ' + wra
    full_article_dict['date'] = datetime
  except:
    full_article_dict['date'] = ''
    pass

  # author
  try:
    author = article.find('span', {'class': 'vima-author'}).text
    full_article_dict['author'] = author
  except:
    full_article_dict['author'] = ''
    pass

  # full_text
  try:
    p_texts_list = []
    paragraphs = article.find('div', {'class': 'post-body main-content pos-rel article-wrapper'}).find_all('p')
    for p in paragraphs:
      p_texts_list.append(p.text)
    full_text = ' '.join(p_texts_list)
    full_text = "".join(full_text.splitlines())
    full_article_dict['full_text'] = full_text
  except:
    full_article_dict['full_text'] = ''
    pass

  full_articles_list.append(full_article_dict)
  time.sleep(1)

In [23]:
tovima_full_articles_df = pd.DataFrame(full_articles_list)
tovima_full_articles_df

,site,url,section,title,date,author,full_text
0,tovima.gr,https://www.tovima.gr/2022/07/30/world/aystria...,Κόσμος,Αυστρία: Αυτοκτόνησε γιατρός που είχε γίνει σ...,30-07-2022 13:46,Newsroom,Οι Αυστριακοί ηγέτες απηύθυναν έκκληση για εθν...
1,tovima.gr,https://www.tovima.gr/2022/01/24/society/agio-...,Κοινωνία,Άγιο Όρος: Θερίζει ο κοροναϊός – Αντιεμβολιασ...,24-01-2022 11:50,Newsroom,"Αντί να εμβολιαστούν κατά του κοροναϊό, μοναχο..."
2,tovima.gr,https://www.tovima.gr/2022/01/23/world/vryksel...,Κόσμος,Βρυξέλλες: Αντιεμβολιαστές βανδάλισαν το κτήρ...,23-01-2022 23:22,Newsroom,Επίθεση διαδηλωτών δέχτηκε το κτήριο της ΕΕ στ...
3,tovima.gr,https://www.tovima.gr/2022/01/08/world/gallia-...,Κόσμος,Γαλλία – Διαδηλώσεις από αντιεμβολιαστές με σ...,08-01-2022 20:59,Newsroom,Αντιεμβολιαστές διαδηλωτές πραγματοποίησαν συγ...
4,tovima.gr,https://www.tovima.gr/2021/12/19/society/masto...,Κοινωνία,Μαστοράκου – Αντιεμβολιαστές την αποκάλεσαν «...,19-12-2021 09:20,Newsroom,"Την πρόθεσή της να προσφύγει στον εισαγγελέα, ..."
5,tovima.gr,https://www.tovima.gr/2021/12/13/society/mitro...,Κοινωνία,Μητροπολίτης Δωδώνης – Πού είναι το μεμπτό ο ...,13-12-2021 13:43,Newsroom,"«Δεν είναι υπερβολή, το να πάει ο πρωθυπουργός..."
6,tovima.gr,https://www.tovima.gr/2021/12/09/inbox/koronoi...,Inbox,Κορωνοϊός – Καταγγελία για δύο αντιεμβολιαστέ...,09-12-2021 19:12,Newsroom,Δεν είναι μόνο ο πόνος της μητέρας που έχασε τ...
7,tovima.gr,https://www.tovima.gr/2021/12/03/vimatv/antiem...,ΒημαTV,Αντιεμβολιαστές – Μαρτυρία στο MEGA – Xωρίζει...,03-12-2021 07:17,Newsroom,Στην εκπομπή Κοινωνία Ώρα MEGA μίλησε η κ. Μαρ...
8,tovima.gr,https://www.tovima.gr/2021/12/01/politics/voul...,Πολιτική,Βουλή – Μητσοτάκης σε Πολάκη – «Ανεβάστε τη μ...,01-12-2021 16:20,Newsroom,"Έντονος διαπληκτισμός επικράτησε στη Βουλή, κα..."
9,tovima.gr,https://www.tovima.gr/2021/12/01/opinions/kai-...,Γνώμες,Και αν οι αντιεμβολιαστές έχουν δίκιο;,01-12-2021 05:50,Κυψελιώτης Διόδωρος,Το θέμα είναι να μην πανικοβληθούμε – γιατί με...


In [24]:
tovima_full_articles_df.to_csv('/content/temp/tovima_full_articles.csv', index=False)